# Práctica 3: Representación del conocimiento

__Fecha de entrega: 23 de febrero de 2023__

El objetivo de esta práctica es aplicar los conceptos teóricos vistos en clase en el módulo de Representación de conocimiento. La práctica consta de un único notebook que se entregará en la tarea de entrega habilitada en el Campus  Virtual.

Cada consulta debe contener breves comentarios que expliquen cada tripleta. __No se valorarán consultas sin explicaciones__.

__Número de grupo: XX__

__Nombres de los estudiantes: ALEJANDRO PAZ OLALLA y ENRIQUE QUEIPO DE LLANO__

## Consultas SPARQL sobre Wikidata.

En esta práctica vamos a usar el [punto de acceso SPARQL](https://query.wikidata.org/) de Wikidata para contestar las preguntas que se formulan a continuación. Cada pregunta debe ser respondida realizando una única consulta SPARQL. Para cada una de las entidades recuperadas se mostrará __tanto su identificador como su etiqueta__ (nombre de la entidad en lenguaje natural). 

Para cada una de las preguntas debes mostrar tanto la consulta como la respuesta obtenida. Si lo consideras necesario, puedes añadir celdas adicionales en formato _Markdown_ para explicar decisiones que hayas tomado al crear la consulta o cualquier otro dato que consideres interesante.
 
Para resolver estas consultas puedes usar __dos recursos que te recomendamos consultar__ son:

- [Este tutorial de SPARQL](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial).
- [Esta recopilación de ejemplos](https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries/examples)

In [1]:
## Lo primero sería instalar en tu entorno de Python wdsparql
## Si no lo tienes instalado, puedes hacerlo desde el notebook
import sys

!{sys.executable} -m pip install wdsparql

# Asumismos instalado wdsparql
%load_ext autoreload
%load_ext wdsparql
%autoreload 2

Defaulting to user installation because normal site-packages is not writeable


### Ejemplo

Recuperar todas las instancias directas de la clase [Cabra (Q2934)](https://www.wikidata.org/wiki/Q2934) que aparecen en la base de conocimiento.

In [2]:
%%wdsparql
SELECT ?item ?itemLabel 
WHERE 
{
  ?item wdt:P31 wd:Q2934.  # instancias directas de la clase Cabra
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

,item,itemLabel
0,http://www.wikidata.org/entity/Q151345,Billygoat Hennes
1,http://www.wikidata.org/entity/Q3569037,William Windsor
2,http://www.wikidata.org/entity/Q13050139,Q13050139
3,http://www.wikidata.org/entity/Q23003932,His Whiskers
4,http://www.wikidata.org/entity/Q24287064,Taffy
5,http://www.wikidata.org/entity/Q41239734,Lance Corporal Shenkin III
6,http://www.wikidata.org/entity/Q41240892,Lance Corporal Shenkin II
7,http://www.wikidata.org/entity/Q41241416,Lance Corporal Shenkin I


### Consulta 1

[U2 (Q396)](https://www.wikidata.org/wiki/Q396) es una banda de rock con una larga trayectoria y numerosos éxitos. 

Vamos a comenzar por averiguar su fecha y lugar de formación (localidad y país).

In [24]:
%%wdsparql
SELECT ?country ?countryLabel ?date ?location ?locationLabel
WHERE 
{
  wd:Q396 wdt:P17 ?country. 
  wd:Q396 wdt:P571 ?date.
  wd:Q396 wdt:P740 ?location.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

,country,countryLabel,date,location,locationLabel
0,http://www.wikidata.org/entity/Q27,Republic of Ireland,1976-01-01T00:00:00Z,http://www.wikidata.org/entity/Q1761,Dublin


### Consulta 2

A continuación vamos a averiguar todos los miembros que se le conocen en la base de conocimiento. Queremos obtener los resultados ordenados alfabéticamente por su apellido e incluir su nombre completo y el artístico.

Puede que la siguiente información sobre el uso de [OPTIONAL](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#OPTIONAL) te resulte útil.

In [35]:
%%wdsparql
SELECT ?member ?memberLabel ?full_name ?artistic ?surnameLabel
WHERE 
{
  wd:Q396 wdt:P527 ?member.
  OPTIONAL {?member wdt:P1477 ?full_name}
  ?member wdt:P734 ?surname
  OPTIONAL {?member wdt:P742 ?artistic}
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ASC(?surnameLabel)

,member,memberLabel,full_name,artistic,surnameLabel
0,http://www.wikidata.org/entity/Q175907,Adam Clayton,Adam Charles Clayton,None,Clayton
1,http://www.wikidata.org/entity/Q714845,The Edge,David Howell Evans,The Edge,Evans
2,http://www.wikidata.org/entity/Q2136220,Dik Evans,None,None,Evans
3,http://www.wikidata.org/entity/Q834621,Bono,Paul David Hewson,Bono,Hewson
4,http://www.wikidata.org/entity/Q814065,Larry Mullen Jr.,Larry Mullen Jr,None,Mullen


Es importante usar dos OPTIONAL distintos para que se muestren todos los miembros, tengan o no pseudónimo y nombre completo asociados.

### Consulta 3

[Bono (Q834621)](http://www.wikidata.org/entity/Q834621) es el cantante de la banda. Además de cantante, ha tenido muchas otras ocupaciones. ¿Cuáles de ellas están relacionadas directamente con el mundo del [arte (Q483501](https://www.wikidata.org/wiki/Q483501)? Ten en cuenta que la jerarquía de ocupaciones puede ser muy sofisticada.

Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#Property_paths) te resulte útil.

In [38]:
%%wdsparql
SELECT ?ocupation ?ocupationLabel
WHERE
{
  wd:Q834621 wdt:P106 ?ocupation.
  ?ocupation wdt:P279+ wd:Q483501
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

,ocupation,ocupationLabel
0,http://www.wikidata.org/entity/Q28389,screenwriter
1,http://www.wikidata.org/entity/Q33999,actor
2,http://www.wikidata.org/entity/Q488205,singer-songwriter
3,http://www.wikidata.org/entity/Q855091,guitarist


Si no añadimos el `+` a la relación `wdt:P279` muchas de las profesiones no se recuperan.

### Consulta 4

Bono se ha casado una vez, con Ali Hewson, con la que tiene varios hijos. Queremos conocer los nombres de esos hijos y sus fechas de nacimiento. Los resultados deben aparecer ordenados cronológicamente.

In [44]:
%%wdsparql
SELECT ?child ?childLabel ?birthDate
WHERE
{
  ?child wdt:P22 wd:Q834621.
  ?child wdt:P25 wd:Q777287.
  ?child wdt:P569 ?birthDate
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ?birthDate

,child,childLabel,birthDate
0,http://www.wikidata.org/entity/Q3735288,Eve Hewson,1991-07-07T00:00:00Z
1,http://www.wikidata.org/entity/Q82782709,Elijah Hewson,1999-08-18T00:00:00Z


### Consulta 5

Bono ha recibido muchos premios a lo largo de su carrera. Queremos obtener la lista de premios y, para cada uno de ellos, la fecha en la que fue premiado (si está disponible). Los resultados se deben mostrar ordenado por fecha.

Para resolver esta consulta necesitarás acceder a los cualificadores de nodos sentencia y necesitarás entender los prefijos que usa Wikidata. Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#Qualifiers) te resulte útil.

In [45]:
%%wdsparql
SELECT ?award ?awardLabel ?awardedDate
WHERE
{
  wd:Q834621 p:P166 ?awardAndDate.
  ?awardAndDate ps:P166 ?award. 
  OPTIONAL {?awardAndDate pq:P585 ?awardedDate}
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ?awardedDate

,award,awardLabel,awardedDate
0,http://www.wikidata.org/entity/Q41254,Grammy Awards,None
1,http://www.wikidata.org/entity/Q52382875,Fellow of the American Academy of Arts and Sci...,None
2,http://www.wikidata.org/entity/Q326482,MusiCares Person of the Year,2003-01-01T00:00:00Z
3,http://www.wikidata.org/entity/Q10855271,Knight of the Legion of Honour,2003-01-01T00:00:00Z
4,http://www.wikidata.org/entity/Q23014208,TED Prize,2005-01-01T00:00:00Z
5,http://www.wikidata.org/entity/Q179191,Rock and Roll Hall of Fame,2005-01-01T00:00:00Z
6,http://www.wikidata.org/entity/Q67155276,Officer of the Order of Liberty,2005-04-21T00:00:00Z
7,http://www.wikidata.org/entity/Q207826,Time Person of the Year,2005-12-19T00:00:00Z
8,http://www.wikidata.org/entity/Q7182737,Philadelphia Liberty Medal,2007-01-01T00:00:00Z
9,http://www.wikidata.org/entity/Q17011862,NAACP Image Award – Chairman's Award,2007-01-01T00:00:00Z


### Consulta 6

Ahora queremos conocer todos los discos de U2 del tipo [albunes en directo (Q209939)](https://www.wikidata.org/wiki/Q209939) y cualquiera de sus subgéneros. Los resultados se deben mostrar ordenados por nombre e incluir el año de publicación.

### Consulta 7

Queremos saber si hay obras de U2 publicadas varias veces (en diferentes años). Para ello vamos a recuperar las obras de tipo [lanzamiento de un grupo (group release)](https://www.wikidata.org/wiki/Q108346082) pero mostrando también la fecha de publicación más antigua de cada una y el número de apariciones. Los resultados deben aparecer en orden cronológico.

Para calcular la fecha correcta tendrás que agrupar las respuestas por album y aplicar una función de agregación sobre las fechas de lanzamiento. Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#Grouping) te resulte útil.

### Consulta 8

U2 es sin duda un grupo con una larga trayectoria. ¿Cuántas discos ha lanzado? Vamos a considerar sólo instancias directas de [album de estudio (Q208569)](https://www.wikidata.org/wiki/Q208569).

### Consulta 9

Entre todos sus discos de estudio, queremos recuperar los títulos, fechas de lanzamiento y discográfica de los discos de la década de los 1990. Si alguna obra tiene varias fechas de publicación, se mostrará si alguna de ellas está dentro de ese periodo.

Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#FILTER) te resulte útil.

### Consulta 10

¿Qué otros personajes famosos nacieron el mismo año que Bono y tiene su misma nacionalidad? Para cada uno de ellos muestra su nombre y ocupación. Muestra los resultados ordenados por nombre y fecha de nacimiento.

### Consulta 11

¿Cuántos asertos hay sobre U2 en Wikidata? Ten en cuenta que puede aparece tanto como sujeto como objeto de cada tripleta.

__Fecha de las consultas: 21 de abril de 2022__